In [1]:
from scipy.io import loadmat                # Biblioteca para carregar arquivos .mat
import numpy as np                     # Biblioteca do numpy para manipulação de matrizes
import mne
from sklearn.preprocessing import LabelEncoder

path = "../../datasets/beta/"

In [2]:
data = np.load(f"{path}data.npy")
labels = np.load(f"{path}labels.npy")
channels = list(np.load(f"{path}channels.npy"))

In [3]:
ch_types = list()

for channel in channels:
    ch_types.append('eeg')

info = mne.create_info(channels, sfreq=250, ch_types=ch_types)
info

<Info | 7 non-empty values
 bads: []
 ch_names: FP1, FPZ, FP2, AF3, AF4, F7, F5, F3, F1, FZ, F2, F4, F6, F8, ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 250.0 Hz
>

In [4]:
unique_labels = sorted(set(labels))

event_dict = dict()

for i in range(len(unique_labels)):
    event_dict[str(unique_labels[i])] = i

In [5]:
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(len(labels), dtype=int),
    le.fit_transform(labels))
)

mne_data = mne.EpochsArray(data, info, events, event_id=event_dict)
# print(mne_data)

filtered_mne_data = mne_data.filter(7, 17)
filtered_mne_data.save("../../datasets/beta/beta_epo.fif", overwrite=True)
# filtered_mne_data

Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 7 - 17 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 17.00 Hz
- Upper transition bandwidth: 4.25 Hz (-6 dB cutoff frequency: 19.12 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 10240 out of 10240 | elapsed:    2.1s finished
